#### S2S centroids plot overplot the shapefile of each watershed
Ming Ge Oct 2023

 Water vapor mixing ratio at 850 hPa (Q850) anomaly normalized anomaly

In [1]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import geoplot as gplt
import xarray as xr
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [2]:
dir_shp = '/glade/u/home/prein/projects/Arizona_WTing/Shapefiles/'
dir_data = '/glade/campaign/mmm/c3we/mingge/COEXIST/AZ_S2S_DATA/'

RegName_1d = ['Arizona West','Arizona East', 'NM North',  'NM South']
n_region = len(RegName_1d)
sWTdata_1d = [None] * n_region
flnm_shp_1d = [None] * n_region

for nr in range(n_region):
    if RegName_1d[nr] == 'Arizona West':
        sWTdata_1d[nr] = dir_data + 'Clusters13514_1501_1982-2018_Q850_JJASO'
        flnm_shp_1d[nr] = dir_shp + 'AZ_West.shp'
    if RegName_1d[nr] == 'Arizona East':
        sWTdata_1d[nr] = dir_data + 'Clusters13514_1502_1982-2018_Q850_JJASO'
        flnm_shp_1d[nr] = dir_shp + 'AZ_East.shp'
    if RegName_1d[nr] == 'NM North':
        sWTdata_1d[nr] = dir_data + 'Clusters13514_HUC6-00_1982-2018_Q850_JJASO'
        flnm_shp_1d[nr] = dir_shp + 'NM_North.shp'
    if RegName_1d[nr] == 'NM South':
        sWTdata_1d[nr] = dir_data + 'Clusters13514_HUC6-03_1982-2018_Q850_JJASO'
        flnm_shp_1d[nr] = dir_shp + 'NM_South.shp'
        

In [3]:
np.linspace(2.0, 3.0, num=5)

array([2.  , 2.25, 2.5 , 2.75, 3.  ])

In [ ]:
proj = ccrs.PlateCarree()
n_wt = 3
# Set up a standard map for latlon data for multiple subplots
fig, ax = plt.subplots(n_wt, n_region,figsize=(15, 12), subplot_kw={'projection': proj})

levels = np.linspace(-2, 2, 9) 

for nr in range(n_region):
    # load shapefile
    shp_gpd = gpd.read_file(flnm_shp_1d[nr])
    
    # Load the Centroids
    with open(sWTdata_1d[nr], 'rb') as handle:
        npzfile = pickle.load(handle)
    
        WTclusters=npzfile['grClustersFin']['Full']
        WTlat=npzfile['LatWT']     
        WTlon=npzfile['LonWT'] 

        # WTlon[WTlon<0] = WTlon[WTlon<0]+360
        lon_1d = WTlon[0, :]
        lat_1d = WTlat[:, 0]

        centroid_3d = np.reshape(WTclusters[0], (WTclusters[0].shape[0],WTlat.shape[0], WTlat.shape[1]))
 
        n_wt = centroid_3d.shape[0]
         
    for nw in range(n_wt):
         # Make a filled contour plot
        filled_c = ax[nw, nr].contourf(lon_1d, lat_1d, centroid_3d[nw], 
                                       levels = levels, cmap = 'YlGn', transform = ccrs.PlateCarree())
        
        # Add coastlines
        ax[nw, nr].coastlines()
        ax[nw, nr].add_feature(cfeature.STATES, zorder=1, linewidth=.3, edgecolor='black')
        ax[nw, nr].set_title(RegName_1d[nr] + ' weather type '+ str(nw))

        # plot the lat lon labels
        #xticks = np.arange(lon_1d.min(), lon_1d.max(), 2) 
        #yticks = np.arange(lat_1d.min(), lat_1d.max(), 2)  
        xticks = np.arange(int(lon_1d.min()), int(lon_1d.max()), 2) 
        yticks = np.arange(int(lat_1d.min()), int(lat_1d.max()), 2)  
        ax[nw, nr].set_xticks(xticks, crs=cartopy.crs.PlateCarree())
        ax[nw, nr].set_yticks(yticks, crs=cartopy.crs.PlateCarree())
        
        # add shapefile outline
        shp_gpd.plot(ax = ax[nw,nr], edgecolor='orange', color='none', linewidth=4)
        ax[nw, nr].set_xlim(lon_1d.min(), lon_1d.max())
        ax[nw, nr].set_ylim(lat_1d.min(), lat_1d.max())

        # Add a colorbar for the filled contour.
        # and pad argument to set colorbar away from x-axis
        #fig.colorbar(filled_c , orientation="horizontal", pad = 0.1);
           
fig.subplots_adjust(right=0.8)
# rect : This parameter is the dimensions [left, bottom, width, height] of the new axes.
cbar_ax = fig.add_axes([0.8, 0.15, 0.03, 0.7])
cbar_ax.set_label('anomaly') #rotation=270)
#fig.suptitle(' Q850 anomaly')
fig.colorbar(filled_c, cax=cbar_ax);